In [1]:
# !pip install wordcloud
# !pip install spacy
# !pip install faiss-cpu

In [2]:
import pandas as pd, re, unicodedata
from collections import Counter
import spacy, re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random
import time, json, sys
import re, unicodedata
from concurrent.futures import ThreadPoolExecutor, as_completed
from groq import Groq
import unicodedata
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
from typing import Sequence, Tuple, List

from dotenv import load_dotenv
import os

load_dotenv()
api1 = os.getenv('llama3_70b_8192')
api2 = os.getenv('gemma2_9b_it')
api3 = os.getenv('llama_instant')
api4 = os.getenv('llama3_8b')

In [3]:
legal_data_path = r".\Datos\sentencias_pasadas.xlsx"

In [4]:
legal_data = pd.read_excel(legal_data_path).drop(columns='Tipo')

In [5]:
legal_data.shape

(329, 7)

In [6]:
legal_data.head(3)

,#,Relevancia,Providencia,Fecha Sentencia,Tema - subtema,resuelve,sintesis
0,1,966965.0,T-185/22,2022-05-31,NaN,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...
1,3,963168.0,T-356/21,2021-10-15,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...
2,5,956201.0,T-351/22,2022-10-07,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...


In [7]:
#https://medium.com/@devbytes/similarity-search-with-faiss-a-practical-guide-to-efficient-indexing-and-retrieval-e99dd0e55e8c

In [8]:
def crear_indice_faiss(
    textos: List[str],
    model_name: str = 'paraphrase-multilingual-mpnet-base-v2'
) -> Tuple[faiss.Index, SentenceTransformer]:

    print("Cargando modelo y creando índice Faiss... (esto puede tardar)")
    model = SentenceTransformer(model_name)
    embeddings = model.encode(textos, show_progress_bar=True).astype('float32')

    # Normalizar vectores es CRUCIAL para que el producto punto sea igual a la similitud coseno
    faiss.normalize_L2(embeddings)
    
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    
    print("¡Índice creado exitosamente!")
    return index, model

def buscar_por_tema(
    df_original: pd.DataFrame,
    topicos: Sequence[str],
    indice_faiss: faiss.Index,
    modelo: SentenceTransformer,
    umbral: float = 0.5
) -> pd.DataFrame:
    """Busca tópicos en el índice y filtra el DataFrame original."""
    
    # Vectorizar solo los tópicos (muy rápido)
    topic_embeddings = modelo.encode(topicos).astype('float32')
    faiss.normalize_L2(topic_embeddings)

    # Buscar en el índice para obtener puntajes de todas las filas
    scores, indices = indice_faiss.search(topic_embeddings, k=indice_faiss.ntotal)
    
    # Encontrar el mejor puntaje para cada fila a través de todos los tópicos
    max_scores = np.zeros(indice_faiss.ntotal)
    for i in range(len(topicos)):
        np.maximum.at(max_scores, indices[i], scores[i])

    # Filtrar, añadir el puntaje y devolver el resultado
    df_resultado = df_original[max_scores >= umbral].copy()
    df_resultado['similarity'] = max_scores[max_scores >= umbral]
    
    return df_resultado.sort_values(by='similarity', ascending=False).reset_index(drop=True)

In [9]:
MODEL_KEYS = [
    ("llama3-70b-8192",      api1),
    ("gemma2-9b-it",         api2),
    ("llama-3.1-8b-instant", api3),
    ("llama3-8b-8192",       api4),
]


MODELS  = [m for m, _ in MODEL_KEYS]
CLIENTS = [Groq(api_key=k) for _, k in MODEL_KEYS]
CALL_INTERVAL = 60 / 29.5

def worker(task):
    idx, content, column = task
    base_slot = idx % len(MODELS)

    prompt = (
        "En español, en una sola oración neutra (≤30 palabras) resume quién pidió qué, el resultado y por qué, tal como aparece en el texto."
        "Usa terminología legal, pero simple sin ser tan sofisticada, pero precisa"
        "Devuelve solo un JSON válido: {\"answer\":\"...\"}. "
        f"Texto: {content}"
    )

    for attempt in range(3):
        slot = (base_slot + attempt) % len(MODELS)
        model, client = MODELS[slot], CLIENTS[slot]

        time.sleep(CALL_INTERVAL)
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_completion_tokens=100,
                response_format={"type": "json_object"},
            )
            result_json = json.loads(resp.choices[0].message.content)
            result = result_json.get("answer", "")
            print(f"✔︎ Model{slot+1} row{idx+1} ({column}) (try{attempt+1})", flush=True)
            return idx, result
        except Exception as e:
            print(f"✖︎ Model{slot+1} row{idx+1} ({column}) failed (try{attempt+1}): {e}",
                  file=sys.stderr, flush=True)

    print(f"⚠︎ Row{idx+1} ({column}) gave up after 3 tries", file=sys.stderr, flush=True)
    return idx, "Failed"

def LLM_aumentation(df):
    tasks = []
    for idx, row in df.iterrows():
        tasks.append((idx, row['resuelve'], "Sentencia_LLM"))
        tasks.append((idx, row['sintesis'], "De_qué_trata_LLM"))

    results_sentencia, results_sintesis = {}, {}

    with ThreadPoolExecutor(len(MODELS)) as pool:
        futures = {pool.submit(worker, task): task for task in tasks}
        for fut in as_completed(futures):
            idx, result = fut.result()
            column = futures[fut][2]
            if column == "Sentencia_LLM":
                results_sentencia[idx] = result
            else:
                results_sintesis[idx] = result

    df["Sentencia_LLM"] = df.index.map(results_sentencia.get)
    df["De_qué_trata_LLM"] = df.index.map(results_sintesis.get)

    return df

In [10]:
columnas_a_buscar = ["Tema - subtema", "resuelve", "sintesis"]

# Pre-procesamos el texto una sola vez
textos_a_indexar = (
    legal_data[columnas_a_buscar]
    .fillna("")
    .astype(str)
    .agg(" ".join, axis=1)
    .str.lower()
    .tolist()
)


indice, modelo = crear_indice_faiss(textos_a_indexar)

Cargando modelo y creando índice Faiss... (esto puede tardar)


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

¡Índice creado exitosamente!


In [11]:
resultados = buscar_por_tema(legal_data, ["red social"], indice, modelo, 0.25)

In [12]:
resultados.shape

(44, 8)

In [13]:
resultados.head(3)

,#,Relevancia,Providencia,Fecha Sentencia,Tema - subtema,resuelve,sintesis,similarity
0,5,956201.0,T-351/22,2022-10-07,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...,0.374928
1,6,955889.0,T-246/21,2021-07-29,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...,0.352689
2,3,963168.0,T-356/21,2021-10-15,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...,0.343402


In [14]:
resultados.sort_values("similarity", ascending=True).head(3)

,#,Relevancia,Providencia,Fecha Sentencia,Tema - subtema,resuelve,sintesis,similarity
43,314,405359.0,A. 486/20,2020-12-15,ESTADO DE COSAS INCONSTITUCIONAL EN MATERIA PE...,RESUELVE: PRIMERO.- ORDENAR al Instituto Nacio...,La Sala Especial de Seguimiento al Estado de C...,0.250719
42,102,783032.0,T-314/24,2024-07-31,NaN,RESUELVE PRIMERO. REVOCAR la Sentencia del 1Â°...,El actor adujo que sus derechos fundamentales ...,0.253419
41,25,927043.0,T-230/20,2020-07-07,ACCION DE TUTELA EN MATERIA DE DERECHO DE PETI...,en nombre del pueblo y por mandato de la Const...,El actor considera que la Empresa de Acueducto...,0.253775


In [15]:
resultados.sort_values("similarity", ascending=True)["sintesis"].iat[0]

'La Sala Especial de Seguimiento al Estado de Cosas Inconstitucional (ECI) en materia penitenciaria y carcelaria declarado en las Sentencias T-388/13 y T-762/15, haciendo uso de su competencia para adoptar medidas de protección de derechos fundamentales en favor de las personas privadas de la libertad, en especial en el contexto de la pandemia actual, ordenó al INPEC diseñar e implementar una categorización de establecimientos de reclusión del orden nacional que se encuentren en riesgo de contagio o con casos activos por COVID 19, en la que se identifiquen dichos lugares según un nivel de seguimiento preventivo, inicial, medio y alto, acorde con los criterios expuestos en la presente providencia, los cuales tienen relación directa con la situación de vulnerabilidad o afectación de dicha población a causa de la emergencia sanitaria. Así mismo, precisó los lineamientos a seguir para dar cumplimiento a la disposición anterior e impartió otra serie de órdenes a varias entidades gubernament

In [16]:
a, b, c = random.sample(range(len(resultados)), 3)
print(a,b,c)

19 15 37


In [17]:
tres_demandas = resultados.loc[[a, b, c]].reset_index(drop=True)
tres_demandas

,#,Relevancia,Providencia,Fecha Sentencia,Tema - subtema,resuelve,sintesis,similarity
0,26,924622.0,T-289/23,2023-08-02,DERECHO A LA PROPIA IMAGEN Y A LA INTIMIDAD-Vu...,"administrando justicia en nombre del pueblo, y...",La accionante actúa como Representante Legal d...,0.285732
1,455,186479.0,C-197/20,2020-06-24,CALAMIDAD PUBLICA-Definición CONSTITUCION POLI...,en nombre del pueblo y por mandato de la Const...,Revisión constitucional automática del Decreto...,0.288448
2,16,937673.0,T-446/20,2020-10-15,ACCION DE TUTELA EN MATERIA DE LIBERTAD DE EXP...,"administrando justicia en nombre del pueblo, y...",En dos acciones de tutela formuladas de manera...,0.259881


In [18]:
tres_demandas_con_LLMs = LLM_aumentation(tres_demandas)
#tres_demandas_con_LLMs.to_csv("legal_cases_with_llm.csv", encoding="utf-8-sig")

✔︎ Model2 row2 (De_qué_trata_LLM) (try1)
✔︎ Model2 row2 (Sentencia_LLM) (try1)
✔︎ Model1 row1 (Sentencia_LLM) (try1)
✔︎ Model1 row1 (De_qué_trata_LLM) (try1)
✔︎ Model3 row3 (Sentencia_LLM) (try1)
✔︎ Model3 row3 (De_qué_trata_LLM) (try1)


In [19]:
tres_demandas_con_LLMs

,#,Relevancia,Providencia,Fecha Sentencia,Tema - subtema,resuelve,sintesis,similarity,Sentencia_LLM,De_qué_trata_LLM
0,26,924622.0,T-289/23,2023-08-02,DERECHO A LA PROPIA IMAGEN Y A LA INTIMIDAD-Vu...,"administrando justicia en nombre del pueblo, y...",La accionante actúa como Representante Legal d...,0.285732,La Sala confirmó la decisión de instancia que ...,"La Fundación Dignidad Trans, representada por ..."
1,455,186479.0,C-197/20,2020-06-24,CALAMIDAD PUBLICA-Definición CONSTITUCION POLI...,en nombre del pueblo y por mandato de la Const...,Revisión constitucional automática del Decreto...,0.288448,Un tribunal resolvía declarar exequible el Dec...,La Corte declaró ejecutable el Decreto Legisla...
2,16,937673.0,T-446/20,2020-10-15,ACCION DE TUTELA EN MATERIA DE LIBERTAD DE EXP...,"administrando justicia en nombre del pueblo, y...",En dos acciones de tutela formuladas de manera...,0.259881,El Tribunal Supremo confirmó la decisión del j...,Las peticionarias solicitaron amparo contra el...


In [20]:
tres_demandas_con_LLMs['Sentencia_LLM'].iloc[0]

'La Sala confirmó la decisión de instancia que declaró improcedente la acción respecto de varias personas, tuteló derechos fundamentales de una menor y ordenó disculpas públicas o privadas a los accionados.'

In [21]:
tres_demandas_con_LLMs['De_qué_trata_LLM'].iloc[0]

'La Fundación Dignidad Trans, representada por la accionante, pidió protección para un grupo de mujeres transgénero cuyos derechos fundamentales fueron vulnerados por la publicación de un video en Facebook sin consentimiento, obteniendo tutela solo para la menor afectada.'